In [1]:
!python -m pip install pyfe3d > tmp.txt

DEPRECATION: Loading egg at /Users/saullogiovanip/miniconda3/lib/python3.12/site-packages/composites-0.7.0-py3.12-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/saullogiovanip/miniconda3/lib/python3.12/site-packages/docopt-0.6.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/saullogiovanip/miniconda3/lib/python3.12/site-packages/coverage-7.5.4-py3.12-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/saull

Creating plate FE model
---

In [1]:
import numpy as np
from numpy import isclose
from scipy.sparse.linalg import eigsh, spsolve, cg
from scipy.sparse import coo_matrix

from pyfe3d.shellprop_utils import isotropic_plate
from pyfe3d import Quad4R, Quad4RData, Quad4RProbe, INT, DOUBLE, DOF


def create_model(thickness=0.003):
    model = {}
    data = Quad4RData()
    probe = Quad4RProbe()
    nx = 50
    ny = 24
    if (nx % 2) == 0:
        nx += 1
    if (ny % 2) == 0:
        ny += 1

    a = 2.0
    b = 0.5

    E = 203.e9 # Pa
    nu = 0.33

    rho = 7.83e3 # kg/m3

    xtmp = np.linspace(0, a, nx)
    ytmp = np.linspace(0, b, ny)
    xmesh, ymesh = np.meshgrid(xtmp, ytmp)
    ncoords = np.vstack((xmesh.T.flatten(), ymesh.T.flatten(), np.zeros_like(ymesh.T.flatten()))).T

    x = ncoords[:, 0]
    y = ncoords[:, 1]
    z = ncoords[:, 2]
    ncoords_flatten = ncoords.flatten()

    nids = 1 + np.arange(ncoords.shape[0])
    nid_pos = dict(zip(nids, np.arange(len(nids))))
    nids_mesh = nids.reshape(nx, ny)
    n1s = nids_mesh[:-1, :-1].flatten()
    n2s = nids_mesh[1:, :-1].flatten()
    n3s = nids_mesh[1:, 1:].flatten()
    n4s = nids_mesh[:-1, 1:].flatten()

    num_elements = len(n1s)

    KC0r = np.zeros(data.KC0_SPARSE_SIZE*num_elements, dtype=INT)
    KC0c = np.zeros(data.KC0_SPARSE_SIZE*num_elements, dtype=INT)
    KC0v = np.zeros(data.KC0_SPARSE_SIZE*num_elements, dtype=DOUBLE)
    KGr = np.zeros(data.KG_SPARSE_SIZE*num_elements, dtype=INT)
    KGc = np.zeros(data.KG_SPARSE_SIZE*num_elements, dtype=INT)
    KGv = np.zeros(data.KG_SPARSE_SIZE*num_elements, dtype=DOUBLE)
    Mr = np.zeros(data.M_SPARSE_SIZE*num_elements, dtype=INT)
    Mc = np.zeros(data.M_SPARSE_SIZE*num_elements, dtype=INT)
    Mv = np.zeros(data.M_SPARSE_SIZE*num_elements, dtype=DOUBLE)

    N = DOF*nx*ny

    prop = isotropic_plate(thickness=thickness, E=E, nu=nu, calc_scf=True, rho=rho)

    quads = []
    init_k_KC0 = 0
    init_k_KG = 0
    init_k_M = 0
    for n1, n2, n3, n4 in zip(n1s, n2s, n3s, n4s):
        pos1 = nid_pos[n1]
        pos2 = nid_pos[n2]
        pos3 = nid_pos[n3]
        pos4 = nid_pos[n4]
        r1 = ncoords[pos1]
        r2 = ncoords[pos2]
        r3 = ncoords[pos3]
        normal = np.cross(r2 - r1, r3 - r2)[2]
        assert normal > 0
        quad = Quad4R(probe)
        quad.n1 = n1
        quad.n2 = n2
        quad.n3 = n3
        quad.n4 = n4
        quad.c1 = DOF*nid_pos[n1]
        quad.c2 = DOF*nid_pos[n2]
        quad.c3 = DOF*nid_pos[n3]
        quad.c4 = DOF*nid_pos[n4]
        quad.init_k_KC0 = init_k_KC0
        quad.init_k_KG = init_k_KG
        quad.init_k_M = init_k_M
        quad.update_rotation_matrix(ncoords_flatten)
        quad.update_probe_xe(ncoords_flatten)
        quad.update_KC0(KC0r, KC0c, KC0v, prop)
        quad.update_M(Mr, Mc, Mv, prop)
        quads.append(quad)
        init_k_KC0 += data.KC0_SPARSE_SIZE
        init_k_KG += data.KG_SPARSE_SIZE
        init_k_M += data.M_SPARSE_SIZE

    KC0 = coo_matrix((KC0v, (KC0r, KC0c)), shape=(N, N)).tocsc()
    M = coo_matrix((Mv, (Mr, Mc)), shape=(N, N)).tocsc()

    bk = np.zeros(N, dtype=bool)
    check = isclose(x, 0.) | isclose(x, a) | isclose(y, 0) | isclose(y, b)
    bk[2::DOF] = check
    # constraining u at x = a/2, y = 0,b
    check = isclose(x, a/2.) & (isclose(y, 0.) | isclose(y, b))
    bk[0::DOF] = check
    # constraining v at x = 0,a y = b/2
    check = isclose(y, b/2.) & (isclose(x, 0.) | isclose(x, a))
    bk[1::DOF] = check
    # removing drilling
    bk[5::DOF] = True

    bu = ~bk

    # applying load along u at x=a
    # nodes at vertices get 1/2 of the force distribution
    fext = np.zeros(N)
    ftotal = -1.
    # at x=0
    check = (isclose(x, 0) & ~isclose(y, 0) & ~isclose(y, b))
    fext[0::DOF][check] = -ftotal/(ny - 1)
    check = ((isclose(x, 0) & isclose(y, 0))
            |(isclose(x, 0) & isclose(y, b)))
    fext[0::DOF][check] = -ftotal/(ny - 1)/2
    assert np.isclose(fext.sum(), -ftotal)
    # at x=a
    check = (isclose(x, a) & ~isclose(y, 0) & ~isclose(y, b))
    fext[0::DOF][check] = ftotal/(ny - 1)
    check = ((isclose(x, a) & isclose(y, 0))
            |(isclose(x, a) & isclose(y, b)))
    fext[0::DOF][check] = ftotal/(ny - 1)/2
    assert np.isclose(fext.sum(), 0)

    Kuu = KC0[bu, :][:, bu]
    fextu = fext[bu]

    PREC = np.max(1/Kuu.diagonal())
    uu, out = cg(PREC*Kuu, PREC*fextu, atol=1e-8)
    assert out == 0, 'cg failed'
    u = np.zeros(N)
    u[bu] = uu

    for quad in quads:
        quad.update_probe_ue(u) # NOTE update affects the Quad4RProbe class attribute ue
        quad.update_probe_xe(ncoords_flatten)
        quad.update_KG(KGr, KGc, KGv, prop)
    KG = coo_matrix((KGv, (KGr, KGc)), shape=(N, N)).tocsc()
    KGuu = KG[bu, :][:, bu]

    model['KC0'] = KC0.copy()
    model['M'] = M.copy()
    model['KG'] = KG.copy()
    model['bu'] = bu.copy()
    model['fext'] = fext.copy()
    model['u0'] = u.copy()
    
    return model

def calculate_buckling(thickness, num_modes=3):
    model = create_model(thickness)
    bu = model['bu']
    Kuu = model['KC0'][bu, :][:, bu]
    KGuu = model['KG'][bu, :][:, bu]
    PREC = np.max(1/Kuu.diagonal())
    eigvals, eigvecsu = eigsh(A=PREC*KGuu, k=num_modes, which='SM',
            M=PREC*Kuu, tol=1e-15, sigma=1., mode='cayley')
    eigvals = -1./eigvals
    load_mult = eigvals[0]
    ftotal = 1.
    P_cr = load_mult*ftotal
    return P_cr

def calculate_natural_frequency(thickness, num_modes=1):
    model = create_model(thickness)
    bu = model['bu']
    Kuu = model['KC0'][bu, :][:, bu]
    Muu = model['M'][bu, :][:, bu]
    eigvals, eigvecsu = eigsh(A=Kuu, M=Muu, sigma=-1., which='LM',
            k=num_modes, tol=1e-3)
    omegan = eigvals**0.5
    return omegan[0]


Optimization
---

In [6]:
from scipy.optimize import minimize, NonlinearConstraint


def volume(thickness):
    a = 2.0
    b = 0.5
    return a*b*thickness

design_load = 80000 # [N]

natural_frequency = NonlinearConstraint(calculate_natural_frequency, 250, np.inf)
buckling = NonlinearConstraint(calculate_buckling, design_load, np.inf)

x0 = [0.003]
thickness_min = 0.001
thickness_max = 0.010
bounds = ((thickness_min, thickness_max),)
out = minimize(volume, x0=x0, constraints=[natural_frequency, buckling], bounds=bounds)

Checking optimization results
---

In [7]:
print(out)
print('natural frequency [rad/s]', calculate_natural_frequency(out.x[0]))
print('critical buckling load [N]', calculate_buckling(out.x[0]))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0038187766380222646
       x: [ 3.819e-03]
     nit: 3
     jac: [ 1.000e+00]
    nfev: 7
    njev: 3
natural frequency [rad/s] 249.99999999895485
critical buckling load [N] 84701.39296638418


In [8]:
0.0028/0.0038-1


-0.26315789473684215